In [1]:
!conda env list

# conda environments:
#
VAdepthENV               /home/013907062/.conda/envs/VAdepthENV
env_onmttf               /home/013907062/.conda/envs/env_onmttf
koen_base                /home/013907062/.conda/envs/koen_base
newDepth                 /home/013907062/.conda/envs/newDepth
test                     /home/013907062/.conda/envs/test
wmt_infer             *  /home/013907062/.conda/envs/wmt_infer
base                     /opt/ohpc/pub/apps/anaconda/3.9
stylegan2                /opt/ohpc/pub/apps/anaconda/3.9/envs/stylegan2



In [2]:
!pip show huggingface_hub

Name: huggingface-hub
Version: 0.17.1
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: /home/013907062/.conda/envs/wmt_infer/lib/python3.10/site-packages
Requires: filelock, fsspec, packaging, pyyaml, requests, tqdm, typing-extensions
Required-by: accelerate, autonlp, datasets, transformers


In [ ]:
!pip install transformers
!pip install datasets
!pip install sacrebleu
!pip install sentencepiece
!pip install accelerate -U

In [ ]:
!pip install transformers -U
!pip install huggingface_hub -U

In [ ]:
!pip install --upgrade accelerate

In [2]:
import pandas as pd
import numpy as np
import multiprocessing
from easydict import EasyDict
import yaml

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

/home/013907062/.conda/envs/wmt_infer/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [5]:
from datasets import load_dataset, load_metric

In [6]:
# Read config.yaml file
with open("config.yaml") as infile:
    SAVED_CFG = yaml.load(infile, Loader=yaml.FullLoader)
    CFG = EasyDict(SAVED_CFG["CFG"])

metric = load_metric("sacrebleu")

# all dataset
#dset = load_dataset(CFG.dset_name, use_auth_token=True)
dset = load_dataset("csv", data_files={'train':'idioms_train.csv','valid': 'idioms__eval.csv'})
print("[LOG YJ]", CFG.model_name)
#QuoQA-NLP/KE-T5-En2Ko-Base
#tokenizer = T5Tokenizer.from_pretrained(CFG.model_name)  # https://github.com/AIRC-KETI/ke-t5#models
tokenizer = AutoTokenizer.from_pretrained("QuoQA-NLP/KE-T5-En2Ko-Base")
print("[LOG YJ]", dset)

/tmp/ipykernel_158539/1705689326.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


[LOG YJ] QuoQA-NLP/KE-T5-En2Ko-Base
[LOG YJ] DatasetDict({
    train: Dataset({
        features: ['en', 'ko'],
        num_rows: 1830
    })
    valid: Dataset({
        features: ['en', 'ko'],
        num_rows: 100
    })
})


In [7]:
def preprocess_function(examples):
    inputs = examples[CFG.src_language]
    targets = examples[CFG.tgt_language]
    model_inputs = tokenizer(inputs, max_length=CFG.max_token_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=CFG.max_token_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# print(preprocess_function(dset["train"].select(range(0, 2))))

CPU_COUNT = multiprocessing.cpu_count() // 2
print("[LOG YJ] CPU_COUNT ", CPU_COUNT)

tokenized_datasets = dset.map(preprocess_function, batched=True, num_proc=CPU_COUNT)
print("[LOG YJ] tokenized_datasets done")

model = AutoModelForSeq2SeqLM.from_pretrained(CFG.model_name)

str_model_name = CFG.model_name.split("/")[-1]
run_name = f"{str_model_name}-finetuned-{CFG.src_language}-to-{CFG.tgt_language}"

print("[LOG YJ] run_name", run_name)

training_args = Seq2SeqTrainingArguments(
    run_name,
    learning_rate=CFG.learning_rate,
    weight_decay=CFG.weight_decay,
    per_device_train_batch_size=CFG.train_batch_size,
    per_device_eval_batch_size=CFG.valid_batch_size,
    evaluation_strategy=CFG.evaluation_strategy,
    # eval_steps=CFG.eval_steps,
    save_steps=CFG.save_steps,
    num_train_epochs=CFG.num_epochs,
    save_total_limit=CFG.num_checkpoints,
    predict_with_generate=True,
    fp16=CFG.fp16,
    gradient_accumulation_steps=CFG.gradient_accumulation_steps,
    logging_steps=CFG.logging_steps,
)

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
print("[LOG YJ] Trainer Ready DONE!")

[LOG YJ] CPU_COUNT  14
[LOG YJ] tokenized_datasets done
[LOG YJ] run_name KE-T5-En2Ko-Base-finetuned-en-to-ko
[LOG YJ] Trainer Ready DONE!


In [ ]:
trainer.train()
trainer.evaluate()
trainer.save_model(CFG.save_path)